In [1]:
import pandas as pd
import os
import re
from miner import TextMiner
import process

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/giovanni/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
param_dic = {
    "host"      : "secop-ds4a-instance.cv1xc7htsshw.sa-east-1.rds.amazonaws.com",
    "database"  : "secopds4a",
    "user"      : "secop_postgres",
    "password"  : "Secopds4a"
}
def connect(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn

In [ ]:
conn = connect(param_dic)

In [13]:
contrato_nit = pd.read_sql('select  count(*) from public.contratos', conn)

In [14]:
contrato_nit

,count
0,9385806


In [7]:
# Create SOCRATA client
from sodapy import Socrata
client = Socrata("www.datos.gov.co",
                 "69DhszChrvwrCK35jdmqonSA2",
                 username="jabernalm@unal.edu.co",
                 password="8A4Z0,u6k2a1")

In [8]:
socrata_dataset_identifier = "xvdy-vvsk"

#results = client.get(socrata_dataset_identifier, content_type="json", query=Query)

metadata = client.get_metadata(socrata_dataset_identifier)
metadata
#secop = pd.DataFrame(pd.DataFrame.from_dict(results))
#secop.head()

{'id': 'xvdy-vvsk',
 'name': 'SECOP I',
 'assetType': 'dataset',
 'attribution': 'Colombia Compra Eficiente',
 'attributionLink': 'https://colombiacompra.gov.co',
 'averageRating': 0,
 'category': 'Gastos Gubernamentales',
 'createdAt': 1572405134,
 'description': 'Información de la plataforma SECOP I',
 'displayType': 'table',
 'downloadCount': 10359,
 'hideFromCatalog': False,
 'hideFromDataJson': False,
 'licenseId': 'CC_40_BY',
 'newBackend': True,
 'numberOfComments': 0,
 'oid': 34486879,
 'provenance': 'official',
 'publicationAppendEnabled': False,
 'publicationDate': 1592866082,
 'publicationGroup': 16671408,
 'publicationStage': 'published',
 'rowsUpdatedAt': 1605259855,
 'rowsUpdatedBy': 'e8qe-xtd4',
 'tableId': 17257600,
 'totalTimesRated': 0,
 'viewCount': 36894,
 'viewLastModified': 1601490532,
 'viewType': 'tabular',
 'approvals': [{'reviewedAt': 1572451280,
   'reviewedAutomatically': True,
   'state': 'approved',
   'submissionId': 2350259,
   'submissionObject': 'publi

In [9]:
socrata_dataset_identifier = "xvdy-vvsk"

Query = """
select 
    count(uid) as conteo, anno_cargue_secop, proponentes_seleccionados
where
    proponentes_seleccionados <> 'No definido'
group by
    proponentes_seleccionados, anno_cargue_secop
limit
    5000000
"""

proponentes = client.get(socrata_dataset_identifier, content_type="json", query=Query)

secop2 = pd.DataFrame(pd.DataFrame.from_dict(proponentes))
secop2.head()

,conteo,anno_cargue_secop,proponentes_seleccionados
0,1,2014,SERVICIOS Y SUMINISTROS ZARO LTDA
1,1,2014,UNIÓN TEMPORAL PEI BILINGÜE
2,1,2014,INVERSIONES CAICEDO RÍOS LTDA
3,1,2012,SEGURIDAD SEAGAL LTDA
4,1,2018,FUNDACIÓN EMPRESARIAL PARA EL DESARROLLO DE YU...


In [10]:
secop2

,conteo,anno_cargue_secop,proponentes_seleccionados
0,1,2014,SERVICIOS Y SUMINISTROS ZARO LTDA
1,1,2014,UNIÓN TEMPORAL PEI BILINGÜE
2,1,2014,INVERSIONES CAICEDO RÍOS LTDA
3,1,2012,SEGURIDAD SEAGAL LTDA
4,1,2018,FUNDACIÓN EMPRESARIAL PARA EL DESARROLLO DE YU...
...,...,...,...
283580,1,2014,UNION TEMPORAL FY ¿ CONSTRUVIAS
283581,1,2011,HAG SA UNICA PROPUESTA
283582,1,2015,ANDIA S.A.S.
283583,1,2012,FUNDACION CENTROS DE APRENDIZAJE NIT. 900.186....


In [16]:
secop2

,conteo,anno_cargue_secop,proponentes_seleccionados
0,1,2014,SERVICIOS Y SUMINISTROS ZARO LTDA
1,1,2014,UNIÓN TEMPORAL PEI BILINGÜE
2,1,2014,INVERSIONES CAICEDO RÍOS LTDA
3,1,2012,SEGURIDAD SEAGAL LTDA
4,1,2018,FUNDACIÓN EMPRESARIAL PARA EL DESARROLLO DE YU...
...,...,...,...
283580,1,2014,UNION TEMPORAL FY ¿ CONSTRUVIAS
283581,1,2011,HAG SA UNICA PROPUESTA
283582,1,2015,ANDIA S.A.S.
283583,1,2012,FUNDACION CENTROS DE APRENDIZAJE NIT. 900.186....


In [15]:
#secop2[secop2.groupby('proponentes_seleccionados')]
secop2.groupby(['proponentes_seleccionados']).count()

,conteo,anno_cargue_secop
proponentes_seleccionados,,
! KABOOM ! DG INSTRUMENTOS Y SONIDO,1,1
&#10146; MELTEC COMUNICACIONES S.A,1,1
&#61558; CONSORCIO MARIA INMACULADA,1,1
&#61558; DOTACION INTEGRAL S.A.S. &#61558; COLTECH S.A.S. &#61558; CALIDAD COMPORTAMIENTOEMPRESARIAL LTDA - Q C E LTDA &#61558; SI S.A. - GRUPOS &#61558; SOEN DOTACIONES EMPRESARIALES LTDA. &#61558; CREACIONES CHAZARI S.A.,1,1
&#61607; AGOFER S.A. &#61607; DISTRIBUIDORA ELECTRICA UNIÓN S.A. &#61607; G Y J FERRETERIAS S.A. &#61607; UNIÓN TEMPORAL EDURBE 2010 &#61607; A VELEZ U Y CIA DEPOSITOS MIRANDA LTDA &#61607; EL CONSTRUCTOR INVESIONES S.A.,1,1
...,...,...
único proponente presentado: COOPERATIVA DE TRANSPORTADORES DEL SARAVITA LTDA COOTRASARAVITA LTDA cuyo NIT N° 890.201.509-9 Representado Legalmente por el Señor LUIS ALFREDO RINCON ALARCON Identificado con cedula de ciudadanía N° 5.745.021 Expedida en San Gil Santander,1,1
único proponente: COMERCIALIZADORA Y DISTRIBUIDORA M-VARGAS,1,1
único proponente: CONSTRUCCIONES ARIAS TORRES E.U. NIT. 900134506-5,1,1


In [17]:
socrata_dataset_identifier = "xvdy-vvsk"

Query = """
select 
    uid, proponentes_seleccionados
where
    proponentes_seleccionados <> 'No definido'
limit
    5000000
"""

proponentes = client.get(socrata_dataset_identifier, content_type="json", query=Query)

secop3 = pd.DataFrame(pd.DataFrame.from_dict(proponentes))
secop3

,uid,proponentes_seleccionados
0,12-1-78618-1007609,GUARDAS DE COLOMBIA LTDA - GUARDACOL LTDA
1,19-11-9783125-9005612,ELI RAMÓN GÓMEZ JARAMILLO
2,12-11-826651-1075444,ANIBAL FRANCO SOTO
3,16-15-5674162-5273950,LUIS ALBERTO CURTIDOR MORENO
4,17-9-432910-6357264,EXPRECARDS S.A.S
...,...,...
354405,11-9-225954-544736,L. M INSTRUMENTS LABORATORIOS ABBOTT B. BRAUN ...
354406,11-15-486892-553679,EUCO LTDA
354407,10-11-423430-620757,UNION TEMPORAL MURO VILLA ROSITA
354408,10-15-376220-0,ARQ. RICARDO ALFONSO SUAREZ PUENTES


In [18]:
df=secop3.groupby(['proponentes_seleccionados']).count()

In [24]:
df[df['uid']>5]

,uid
proponentes_seleccionados,
&#61656; PHARMEUROPEA DE COLOMBIA con NIT: 830088135-5 &#61656; QUIRUMEDICAS LTDA con NIT: 86004763-5 &#61656; DRAEGER COLOMBIA S.A con NIT: 900194910-4 &#61656; ASCAVI GROUPS S.A.S con NIT: 900201763-9 &#61656; INGENIERIA CLINICA A SU SERVICIO INCLSER LTDA con NIT: 900180069-3 &#61656; JALSOSA COLOMBIA con NIT: 900742582-3 &#61656; ALFA TRADING SAS con NIT: 830.041.488-7 &#61656; UNION MEDICAL S.A.S con NIT: 811.039.981-1 &#61656; LABORATORIOS LTDA DE BOGOTA con NIT: 890922891-1 &#61656; LABORATORIOS GOTHAPLAST LTDA con NIT: 830-061-856-1 &#61656; 3 M COLOMBIA S.A con NIT: 860.002.693-3 &#61656; RONELLY S.A con NIT: 890929073-5 &#61656; MEDICAL CARE WALL SAS con NIT: 900170805-5 &#61656; ORDENES Y SUMINISTROS SAS con NIT: 900.147.252-6 &#61656; UNION TEMPORAL ELEMENTOS DE CURACION &#61656; LM INSTRUMENTS S.A con NIT: 80077635-1 &#61656; IMCOLMEDICAS S.A con NIT: 860.070.078-3 &#61656; TECNICA ELECTROMIDICA con NIT: 830.004.892-2 &#61656; EUROETIKA con NIT: 800.110.385-4 &#61656; INCOMEDIS LTDA con NIT: 800.091.472-4 &#61656; INDUSTRIAS CARDIOMED SAS con NIT: 800.119.669-1 &#61656; MEDITEC S.A con NIT: 860.038.579-7 &#61656; ABBOTT LABORATORIES DE COLOMBIA S.A con NIT: 860.002.134-8 &#61656; PROTEX S.A con NIT: 830.051.740-1 &#61656; BIOPLAST S.A con NIT: 830.040.574-8 &#61656; SURGICON Y CIA S.A con NIT: 860.519.267-9 &#61656; VEYALTH GROUP S.A.S con NIT: 830.119.211-1 &#61656; M&M EQUIPOS MEDICOS S.A.S con NIT: 830.146.016-6 &#61656; QUIRURGICOS LTDA com NIT: 860.024.862-6 &#61656; SHERLEG LABORATORIES SAS con NIT: 900.539.662-5 &#61656; LIFE SUMINISTROS MEDICOS S.A.S con NIT: 900-240-177-9 &#61656; COBOS Y ASOCIADOS S.A.S con NIT: 890-331-949-3 &#61656; LABORATORIOS BAXTER S.A con NIT: 890300292-0 &#61656; PROASEPSIS S.A.S con NIT: 8605340454-3,25
-,41
- CONSORCIO COSEING LTDA P.I. S.A.S. - CIS CONSTRUCCIONES SERVICIOS E INGENIERIA EU. - CONSORCIO INTERCARCEL. - CONSORCIO GAP. - YOHAN FAHIR BERMUDEZ PARRA. - JORGE ALVARO SANCHEZ BLANCO. - CONSTRUOBRAS C&M S.A.S.,12
- Construobras C&M S.A.S. - Construobras C&M S.A.S. - Construobras C&M S.A.S. - INTEC Ingenieros Civiles y Arquitectos Ltda. - Gustavo Palacios Rubiano - Gustavo Palacios Rubiano - Rodrigo Antonio Arias Chaustre - Rodrigo Antonio Arias Chaustre - SODIARK Construcciones S.A.S. - Consorcio Anabel - Consorcio Anabel - German Alfredo Bazzani Pradera - Consorcio GAP - AN Construdiseño S.A.S. - Gordillo & Asociados S.A.S. - Consorcio GAP - SODIARK Construcciones S.A.S. - AN Construdiseño S.A.S. - Gordillo & Asociados S.A.S. - Consorcio GAP - SODIARK Construcciones S.A.S.,21
- GRUPO 1: CONSORCIO MANTENIMIENTO CARCELES 2015. - GRUPO 2: CONSORCIO WC. - GRUPO 5: SOCIEDAD ESPECIALIZADA EN INGENIERIA SENING S.A.S. - GRUPO 6: INGENIERIA ESPECIALIZADA GESTION AMBIENTAL SEGURIDAD INDUSTRIAL S.A.S. - GRUPO 7: CONSORCIO WC. - GRUPO 8: INVERSIONES CEAC LTDA. - GRUPO 9: JOSE JAVIER MORENO PACHON. - GRUPO 10: CONSORCIO IDAVIG.,8
...,...
ÌTEM 1 INSUMOS DE MODA ITEM 2 INSUMOS DE MODA ITEM 3 INSUMOS DE MODA ITEM 4 AMPLAS S.A. ITEM 5 INDUSTRIAS REAL ITEM 6 UT FEXCOL ITEM 7 C.I TEXMAN LTDA ITEM 8 UT FEXCOL ITEM 9 UT FEXCOL ITEM 10 KROMIA S.A. ITEM 11 UT FEXCOL ITEM 12 UT FEXCOL ITEM 13 UT FEXCOL ITEM 14 FABRILAR LTDA ITEM 16 UT FEXCOL ITEM 18 UT FEXCOL ITEM 19 FABRILAR LTDA ITEM 20 CI TEXMAN LTDA,7
ÍTEM 1 AMPLAS SECURITY S.A.S ITEM 2 DECLARADO DESIERTO ITEM 3 UNIÓN TEMPORAL DOTAMUNDO 154 ITEM 4 FABRILAR LTDA ITEM 5 DECLARADO DESIERTO ITEM 6 EL COMERCIO DEL CUERO LTDA ITEM 7 INVERSIONES GAO LTDA ITEM 8 DECLARADO DESIERTO ITEM 9 AMPLAS SECURITY S.A.S ITEM 10 UNIÓN TEMPORAL NF 2015,6
ÍTEM PROPONENTE REPRESENTANTE LEGAL CC. No. 1 MULTICONSECIONARIO S.A.S. ¿ GRUPO 1 LUIS ALBERTO GALVIS DIAZ 14.220.520 de Ibagué ¿ Tolima 2 SANDRA YANET DIAZ JIMENEZ (DISTRIREPUESTOS) ¿ GRUPO 2 SANDRA YANET DIAZ JIMENEZ 52.551.967 de Engativá 3 LUIS FERNANDO ROBAYO DIAZ (MAUTO VEHICULOS) GRUPO 3 LUIS FERNANDO ROBAYO DIAZ 14.224.583 de Ibagué ¿ Tolima 4 SANDRA YANET DIAZ JIMENEZ (D